### the following is heavily influenced by this [blogpost](http://adventuresinmachinelearning.com/word2vec-tutorial-tensorflow/)

In [1]:
#import data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#data types
import collections

#import general libraries
import gc
import time
import random
import datetime

#deep learning 
import tensorflow as tf

/Users/waficel-assi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [106]:
#list_of_products = pd.read_csv('./data/subset_products_list_july30.csv')
#df_product_embeddings = pd.read_csv('./data/subset_data_for_embeds_july30.csv')
df_products_orders_all = pd.read_csv('./data/df_products_orders_all.csv')

EmptyDataError: No columns to parse from file

In [3]:
df_products_orders_all.head()

,user_id,order_id,order_number,add_to_cart_order,product_id,product_name
0,1.0,473747.0,3.0,1.0,196,Soda
1,1.0,473747.0,3.0,2.0,12427,Original Beef Jerky
2,1.0,473747.0,3.0,3.0,10258,Pistachios
3,1.0,473747.0,3.0,4.0,25133,Organic String Cheese
4,1.0,473747.0,3.0,5.0,30450,Creamy Almond Butter


In [27]:
count_product_user = df_product_embeddings.groupby(['user_id','order_id']).agg({'product_id':'count'})

In [31]:
print(count_product_user.product_id.mean())
print(count_product_user.product_id.median())

10.085320529540459
8.0


In [ ]:
count_product_user_unique = df_product_embeddings.groupby('user_id').agg({'product_id':'nunique'})
count_product_user_unique.product_id.describe()

In [33]:
list_of_products = df_product_embeddings.product_name.tolist()

In [34]:
#number of unique products in dataset
print(len(set(list_of_products)))
n_products = len(set(list_of_products))

49688


In [35]:
#build skip-gram dataset
def embeddings_pre_processing(categories, n_categories):
    """Process raw inputs into a dataset."""
    
    count = [['UNK', -1]]
    count.extend(collections.Counter(categories).most_common(n_categories - 1))
    most_common_category_dict = dict()
    for category, _ in count:
        most_common_category_dict[category] = len(most_common_category_dict)
    
    idx_categories = list()
    unk_count = 0
    for category in categories:
        if category in most_common_category_dict:
            index = most_common_category_dict[category]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        idx_categories.append(index)
    
    count[0][1] = unk_count
    reversed_most_common_category_dict = dict(zip(most_common_category_dict.values(),
                                                   most_common_category_dict.keys()))
    
    return idx_categories, count, most_common_category_dict, reversed_most_common_category_dict

In [36]:
idx_products, count_products, most_common_products_dict, reversed_most_common_products_dict = embeddings_pre_processing(list_of_products, n_products)

In [92]:
reversed_most_common_products_dict

{0: 'UNK',
 1: 'Banana',
 2: 'Bag of Organic Bananas',
 3: 'Organic Strawberries',
 4: 'Organic Baby Spinach',
 5: 'Organic Hass Avocado',
 6: 'Organic Avocado',
 7: 'Large Lemon',
 8: 'Strawberries',
 9: 'Limes',
 10: 'Organic Whole Milk',
 11: 'Organic Raspberries',
 12: 'Organic Yellow Onion',
 13: 'Organic Garlic',
 14: 'Organic Zucchini',
 15: 'Organic Blueberries',
 16: 'Cucumber Kirby',
 17: 'Organic Fuji Apple',
 18: 'Organic Lemon',
 19: 'Apple Honeycrisp Organic',
 20: 'Organic Grape Tomatoes',
 21: 'Seedless Red Grapes',
 22: 'Organic Cucumber',
 23: 'Honeycrisp Apple',
 24: 'Organic Baby Carrots',
 25: 'Organic Half & Half',
 26: 'Organic Large Extra Fancy Fuji Apple',
 27: 'Sparkling Water Grapefruit',
 28: 'Carrots',
 29: 'Yellow Onions',
 30: 'Organic Gala Apples',
 31: 'Organic Baby Arugula',
 32: 'Fresh Cauliflower',
 33: 'Original Hummus',
 34: 'Organic Cilantro',
 35: 'Half & Half',
 36: 'Organic Small Bunch Celery',
 37: 'Michigan Organic Kale',
 38: 'Organic Red On

In [37]:
data_index = 0
# generate batch data
def generate_batch(data, batch_size, num_skips, skip_window):
    
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    
    return batch, context


In [77]:
batch_size = 300
embedding_size = 300  # Dimension of the embedding vector.
skip_window = 3       # How many words to consider left and right.
num_skips = 1         # How many times to reuse an input to generate a context.
products_size = n_products

In [78]:
valid_size = 5     # Random set of words to evaluate similarity on.
valid_window = 50  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [79]:
tf.reset_default_graph()
tf.get_default_graph()

In [80]:
graph = tf.Graph()
num_sampled = 5000

In [81]:
with graph.as_default():
        
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    embeddings = tf.get_variable('embeddings_graph', [products_size, embedding_size])
    embed_product_ids = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    # Construct the variables for the NCE loss
    nce_weights = tf.get_variable('nce_weights', [products_size, embedding_size])
    nce_biases = tf.get_variable('biases', [products_size], initializer=tf.zeros_initializer)

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed_product_ids,
                       num_sampled=num_sampled,
                       num_classes=products_size))

    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)

    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    # Add variable initializer.
    init = tf.global_variables_initializer()

In [82]:
# #create embeddings variable and lookup tensor
# embeddings = tf.get_variable('embeddings_y', [20000, embedding_size])
# embed_product_ids = tf.nn.embedding_lookup(embeddings, train_inputs)

# #create weights and biases
# weights = tf.get_variable('weights_embeds', [20000, embedding_size])
# biases = tf.get_variable('biases', [20000], initializer=tf.zeros_initializer)
# hidden_out = tf.matmul(embed_product_ids, tf.transpose(weights)) + biases

# # convert train_context to a one-hot format
# train_one_hot = tf.one_hot(train_labels, 20000)
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hidden_out, labels=train_one_hot))
# # Construct the SGD optimizer using a learning rate of 0.1.
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cross_entropy)

# norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
# normalized_embeddings = embeddings / norm
# valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
# similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

# init = tf.global_variables_initializer()


In [83]:
def get_embeddings(graph, num_steps):
    with tf.Session(graph = graph) as session:
    
        # We must initialize all variables before we use them.
        init.run()
        print('Initialized')

        #tf.global_variables_initializer().run()

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(idx_products, batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, nce_loss], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                  # The average loss is an estimate of the loss over the last 2000 batches.
                print(' ')
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0
                
            if step % 4000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = reversed_most_common_products_dict[valid_examples[i]]
                    top_k = 5  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = reversed_most_common_products_dict[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                    print(log_str)
                    print(' ')


        final_embeddings = normalized_embeddings.eval()
        return final_embeddings

In [85]:
%%time
product_embeddings_results_10000 = get_embeddings(graph, num_steps=12000)

Initialized
Average loss at step  0 :  23417.6328125
Nearest to Organic Blueberries: Sliced Sharp Cheddar Cheese, Organic Raspberries, Organic Avocado, Organic Old Fashioned Rolled Oats, Organic Muenster Cheese Slices,
Nearest to Limes: Excellence Non-Drip Crème  01 Extra Light Ash Blonde, Gluten Free Everything Baked Cracker, Go-Gurt SpongeBob SquarePants Strawberry Riptide/Sponge Berry Variety Pack Portable Low Fat Yogurt, Dijon Mustard, Tam Tams Everything Snack Crackers,
Nearest to Unsalted Butter: Nilla Wafers, Fruit Snacks Banana, Cherry, Peach & Strawberry Snack Packs, Refried Black Beans Low Fat, Organic Moroccan Chermoula, Bucatini, No. 15,
Nearest to Red Onion: Oven Fry Seasoned Extra Crispy Pork Coating Mix, Balsamic Vinegar of Modena Aged, Sensitive Facial Cleanser, Praline Chocolate Bar, Organic Red Lentil Penne,
Nearest to Organic Tomato Cluster: Milano Double Chocolate Cookies, Organic Jammy Sammy Blueberry & Oatmeal Sandwich Bars, Original Value Pack Sausage Patties, Wo

Nearest to Organic Blueberries: Spring Water, Fresh Cauliflower, Organic Broccoli, Unsweetened Vanilla Almond Milk, Organic Original Almond Milk,
Nearest to Limes: Organic Yellow Onion, Asparagus, Honeycrisp Apple, Organic Zucchini, Uncured Genoa Salami,
Nearest to Unsalted Butter: Jazz Apple, Calorie Free Sweetener, Salted Tub of Butter, Vanilla Skyr Nonfat Yogurt, Blueberry Oats and Quinoa Whole Grain Snack,
Nearest to Red Onion: Avocado Oil, Shredded Parmesan Cheese, Heavy Whipping Cream, Natural Deli Style Sliced Swiss Cheese, Gluten Free Whole Grain Bread,
Nearest to Organic Tomato Cluster: Wild Collection Bearglove Men's Deodorant, Donettes Powdered Mini Donuts, IncrediBites for Small Dogs, Toasted Coconut Almondmilk Blend, Organic Garnet Sweet Potato (Yam),
Nearest to Large Lemon: Spring Water, Sour Cream, Organic Fuji Apple, Organic Muenster Cheese Slices, Almond Nut & Rice Cracker Snacks,
Nearest to Boneless Skinless Chicken Breasts: Lightly Salted Tortilla Strips, Natural Bee

Nearest to Organic Blueberries: Apple Honeycrisp Organic, Organic Whole Milk, Organic Cucumber, Organic Large Extra Fancy Fuji Apple, Organic Lemon,
Nearest to Limes: Organic Zucchini, Large Lemon, Honeycrisp Apple, Organic Garlic, Organic Lemon,
Nearest to Unsalted Butter: Butterhead Lettuce, Fruit Snacks Banana, Cherry, Peach & Strawberry Snack Packs, Vanilla Skyr Nonfat Yogurt, Organic Nectarine, Tall Kitchen Bags, Handle-Tie, 13 Gal,
Nearest to Red Onion: Avocado Oil, Grade A Large Eggs Cage Free Omega 3, Shredded Parmesan Cheese, White Onion, Heavy Whipping Cream,
Nearest to Organic Tomato Cluster: Organic Garnet Sweet Potato (Yam), Michigan Organic Kale, Organic Ginger Root, Organic Lacinato (Dinosaur) Kale, Organic Unsweetened Almond Milk,
Nearest to Large Lemon: Honeycrisp Apple, Organic Fuji Apple, Limes, Organic Garlic, Organic Zucchini,
Nearest to Boneless Skinless Chicken Breasts: Yellow Onions, Red Vine Tomato, Feta Cheese Crumbles, Red Peppers, Organic Reduced Fat Milk,
N

In [86]:
product_embeddings_results_10000[1]

array([-0.06632879,  0.10486741,  0.02446856, -0.03373519, -0.05006214,
        0.06240219,  0.01618789, -0.02983681, -0.00649585, -0.04440806,
        0.01049638, -0.00693365,  0.08074813,  0.00945919,  0.00813178,
       -0.01128302, -0.02985205, -0.12385998, -0.03963036,  0.04559349,
       -0.00109104,  0.0579421 , -0.01823827, -0.08067311,  0.03256713,
       -0.05371716,  0.03773097, -0.02481837, -0.00472766, -0.04100472,
       -0.06333721,  0.024319  , -0.15682925, -0.10684592, -0.09458172,
        0.01826714, -0.10020502, -0.07132937, -0.01144975, -0.06021092,
        0.00481475, -0.0246927 , -0.04807175,  0.00941301,  0.05228074,
        0.01126312,  0.04577968, -0.03546591,  0.03060435,  0.08888792,
       -0.00956132,  0.16163878,  0.03221595,  0.00321678,  0.03099353,
       -0.00590089,  0.03108284,  0.01953761, -0.00383407, -0.01192917,
       -0.0959786 ,  0.00543824,  0.03870308, -0.04003516, -0.02897525,
       -0.05300719,  0.07184929,  0.07362343,  0.13805352, -0.01

In [100]:
index_df = pd.DataFrame.from_dict(reversed_most_common_products_dict,orient='index')
index_df.columns = ['product_name']

embeddings_w_product_name = pd.concat([index_df, pd.DataFrame(product_embeddings_results_10000)], axis=1)

In [102]:
embeddings_w_product_name.to_csv('./data/sample__product_embeddings_results_10000.csv', index=False)